In [1]:
import sys
sys.path.append('C://Users/11max/PycharmProjects/OpenIOCanada/')
import openIO
import pandas as pd
import numpy as np
import gzip
import pickle

Load the pickle file you saved from running openIO-Canada once (see the Running_openIO-Canada.ipynb file if you haven't yet)

In [2]:
with gzip.open('C://Users/11max/PycharmProjects/OpenIOCanada/v2.10/openIO_2021_endogenized.gz.pickle','rb') as f:
    tables = pd.read_pickle(f)

## Generic results

Total impacts of Canada are provided by the D matrix

To calculate the total impacts of Canada following a **_consumption approach_**, we simply need to exclude the impacts of exported commodities/services

In [4]:
tables.D.drop('International exports', axis=1, level=1)

CA-AB  \
                                                                                                           Changes in inventories   
                                                                      Changes in inventories, finished goods and goods in process   
Climate change, ecosystem quality, long term       PDF.m2.yr                                               -1.194969e+09            
Climate change, ecosystem quality, short term      PDF.m2.yr                                               -3.266590e+08            
Climate change, human health, long term            DALY                                                    -5.442052e+03            
Climate change, human health, short term           DALY                                                    -1.511084e+03            
Climate change, long term                          kg CO2 eq (long)                                        -2.064240e+09            
...                                                                                                                  ...            
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                      -4.675188e+00            
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                      -4.675188e+00            
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                      -4.675188e+00            
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                      -4.675188e+00            
Physical effects on biota                          PDF.m2.yr                                               -7.408614e+03            

                                                                                                                                            \
                                                                                                                                             
                                                                      Changes in inventories, raw materials and goods purchased for resale   
Climate change, ecosystem quality, long term       PDF.m2.yr                                                9.475703e+08                     
Climate change, ecosystem quality, short term      PDF.m2.yr                                                2.883244e+08                     
Climate change, human health, long term            DALY                                                     4.314042e+03                     
Climate change, human health, short term           DALY                                                     1.332660e+03                     
Climate change, long term                          kg CO2 eq (long)                                         1.555190e+09                     
...                                                                                                                  ...                     
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                       6.328101e+00                     
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                       6.328101e+00                     
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                       6.328101e+00                     
Disposed plastic waste and scrap mismanaged (op... tonnes of plastics                                       6.328101e+00                     
Physical effects on biota                          PDF.m2.yr                                                8.776631e+04                     

                                                                                                                                 \
                                                                                                                                  
          

If we sum on the columns (i.e., do not care about which final demand is responsible) and we divide by the total population of Canada (~38 million), we get the per capita impact of Canadians (example with the GWP100 value)

In [5]:
tables.D.drop('International exports', axis=1, level=1).sum(axis=1).loc[('Climate change, short term','kg CO2 eq (short)')] / 38000000

19501.90937741305

Now say we want the same for Quebecers, we select only the Quebec final demands and divide by the population of Quebec (~8.5 million)

In [6]:
tables.D.drop('International exports', axis=1, level=1).loc[:,'CA-QC'].sum(axis=1).loc[('Climate change, short term','kg CO2 eq (short)')] / 8500000

15870.049811220528

If you are rather interested by the **_territorial approach_**, you can simply take the values from the F and FY matrices and multiply them by the C matrix

In [7]:
(tables.C.dot(tables.F).sum(axis=1) + tables.C.dot(tables.FY).sum(axis=1)).loc[('Climate change, short term','kg CO2 eq (short)')] / 38000000

19894.452724230716

In [10]:
(tables.C.dot(tables.F).loc[:,'CA-QC'].sum(axis=1) + tables.C.dot(tables.FY).loc[:,'CA-QC'].sum(axis=1)).loc[('Climate change, short term','kg CO2 eq (short)')] / 8500000

10550.827023523165

## More analysis

Say we want to calculate the Quebec household consumption footprint for their direct consumption (excluding government and capital footprint that should be attributed to households).

In [11]:
sector = ('CA-QC','Household final consumption expenditure')

We first diagonalize the demand

In [12]:
diag_Y_households = pd.DataFrame(np.diag(tables.Y.loc[:, sector].sum(1)),
                                                index=tables.Y.index, columns=tables.Y.index)

Then we get the contribution of each purchase by doing the following operation

In [13]:
contribution_analysis_households = tables.C.dot(tables.S).dot(tables.L).dot(diag_Y_households)

This only covered indirect emissions of the households. We have to add the direct emissions as well (e.g., using one's car)

In [14]:
direct_emissions = tables.C.dot(tables.FY)[tables.C.dot(tables.FY)!=0].dropna(how='all',axis=1).fillna(0)
direct_emissions.columns = direct_emissions.columns.droplevel(1)
direct_emissions.columns = pd.MultiIndex.from_product([direct_emissions.columns.levels[0], [
    "Private vehicle use","Heating and cooking","Water use by households"]])

We add both together

In [15]:
contribution_analysis_households = pd.concat([contribution_analysis_households, 
                                              direct_emissions.drop([
                                                  i for i in direct_emissions.columns.levels[0] if i != 'CA-QC'],axis=1)],
                                             axis=1)

And now we get the results (example with GWP100)

In [17]:
contribution_analysis_households.loc[('Climate change, short term','kg CO2 eq (short)')].T.sort_values(ascending=False).iloc[:25]

CA-QC  Private vehicle use                                                          1.492214e+10
       Residential buildings                                                        1.181995e+10
       Heating and cooking                                                          7.513357e+09
       Motor gasoline                                                               3.821458e+09
       Prepared meals                                                               3.460293e+09
US     Motor vehicles, trailers and semi-trailers (34)                              3.027666e+09
CA-QC  Rental of residential real estate                                            2.757943e+09
WA     Wearing apparel; furs (18)                                                   2.245460e+09
CA-QC  Imputed rental of owner-occupied dwellings                                   2.006033e+09
US     Meat animals nec                                                             1.553657e+09
CA-QC  Retail margins - food a

In relative terms might be easier to interpret:

In [24]:
(contribution_analysis_households.loc[('Climate change, short term','kg CO2 eq (short)')]/
 contribution_analysis_households.loc[('Climate change, short term','kg CO2 eq (short)')].sum()*100).sort_values(ascending=False).iloc[:25]

CA-QC  Private vehicle use                                                          13.109366
       Residential buildings                                                        10.384038
       Heating and cooking                                                           6.600620
       Motor gasoline                                                                3.357219
       Prepared meals                                                                3.039929
US     Motor vehicles, trailers and semi-trailers (34)                               2.659859
CA-QC  Rental of residential real estate                                             2.422903
WA     Wearing apparel; furs (18)                                                    1.972677
CA-QC  Imputed rental of owner-occupied dwellings                                    1.762336
US     Meat animals nec                                                              1.364915
CA-QC  Retail margins - food and beverages                  

If you don't care about the origin of commodities/services

In [27]:
(contribution_analysis_households.loc[('Climate change, short term','kg CO2 eq (short)')]/
 contribution_analysis_households.loc[('Climate change, short term','kg CO2 eq (short)')].sum()*100).groupby(axis=0, level=1).sum().sort_values(ascending=False).iloc[:25]

Private vehicle use                                                          13.109366
Residential buildings                                                        10.384038
Heating and cooking                                                           6.600620
Motor vehicles, trailers and semi-trailers (34)                               4.791634
Motor gasoline                                                                3.690732
Wearing apparel; furs (18)                                                    3.568322
Prepared meals                                                                3.518263
Rental of residential real estate                                             2.431845
Imputed rental of owner-occupied dwellings                                    1.762336
Chemicals nec                                                                 1.524246
Meat animals nec                                                              1.413175
Fresh and frozen beef and veal             